## Testing the ability to distinguish signal events from background

In [1]:
%pylab inline
import sys
sys.path.insert(0, '..')

Populating the interactive namespace from numpy and matplotlib


In [2]:
from data import Dataset
dataset = Dataset('../data/signal_TDR.root')

In [3]:
labels = numpy.vstack(dataset.get_hit_types(i) for i in range(dataset.n_events))
# signal vs other
labels = labels == 1

In [4]:
deposits = numpy.vstack(dataset.get_energy_deposits(i) for i in range(dataset.n_events))

In [5]:
labels.shape, deposits.shape

((3445, 4482), (3445, 4482))

## Preapring features

In [6]:
# to avoid small numbers for GBRT
deposits *= 100

In [7]:
avn_deposits = dataset.wire_neighbours.dot(deposits.T).T
avn_deposits_count = dataset.wire_neighbours.dot(deposits.T ** 0.1).T

In [8]:
avn_deposits2 = dataset.wire_neighbours.dot(avn_deposits.T).T

## Train/test splitting 

In [9]:
from sklearn.cross_validation import train_test_split
train_events, test_events = train_test_split(range(len(labels)), train_size=0.05, test_size=0.05)

In [10]:
from collections import OrderedDict
features = OrderedDict()
features['deposit'] = deposits * 100
features['av_deposit'] = avn_deposits
features['av_deposits_count'] = avn_deposits_count
features['av_deposits_twice'] = avn_deposits2
features['labels'] = labels

In [11]:
train_features = set(features.keys())
train_features.remove('labels')

In [12]:
train_features

{'av_deposit', 'av_deposits_count', 'av_deposits_twice', 'deposit'}

In [13]:
data = numpy.dstack(features.values())

In [14]:
n_feats = data.shape[-1]
train_data, test_data = data[train_events].reshape(-1, n_feats), data[test_events].reshape(-1, n_feats)

In [15]:
import pandas
train_data = pandas.DataFrame(data=train_data, columns=features.keys())
test_data = pandas.DataFrame(data=test_data, columns=features.keys())

## Using simple GBRT to determine if wire is signal

In [16]:
from rep.metaml.factory import ClassifiersFactory
from rep.estimators import SklearnClassifier
from sklearn.ensemble import GradientBoostingClassifier

ImportError: No module named rep.metaml.factory

In [ ]:
factory = ClassifiersFactory()
gb_all_features = SklearnClassifier(GradientBoostingClassifier(n_estimators=40, learning_rate=0.1), 
                                    features=train_features)
gb_deposit = SklearnClassifier(GradientBoostingClassifier(n_estimators=40, learning_rate=0.1), 
                               features=['deposit'])
factory.add_classifier('gb', gb_all_features)
factory.add_classifier('gb_deposit', gb_deposit)

In [ ]:
factory.fit(train_data, train_data['labels'])
pass

In [ ]:
predictions = factory.test_on(test_data, test_data['labels'])

In [ ]:
from rep.report.metrics import RocAuc
predictions.learning_curve(RocAuc(), steps=1)

In [ ]:
predictions.roc()

In [ ]:
predictions.features_correlation_matrix().plot(new_plot=True, show_legend=False, figsize=(15, 5))

In [ ]:
predictions.feature_importance().plot(figsize=[14, 7])

In [ ]:
predictions.features_pdf(features=['log(0.001 + deposit)'])